# TME 6 : Mini-projet Detection de motifs


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons implémenter deux algorithmes randomisés pour la recherche de motifs : Greedy Profile Motif Search et random Projections.


1\. Nous allons réutiliser les fonctions du TME5 pour générer t séquences artificielles de taille n, et implanter dans chaque séquence un motif de taille k à des positions aléatoires avec v substitutions choisies aléatoirement.

In [2]:
import random
import numpy as np
import time

nuc = ('A', 'C', 'G', 'T')

k=7 #taille de motif
v=4 #nb de positions variable dans le motif
t=20 #nb de sequences
n=50 #longuer des sequence

def generateRandomSequences(n, t):
    sequences = []
    i = 0
    j = 0
    while i < t:
        seq = ""
        while j < n:
            seq = seq+nuc[random.randint(0,3)]
            j += 1
        sequences.append(seq)
        i += 1
        j = 0
   
    return sequences

# RENVOIE UNE LISTE DE LISTE DE MOTIFS...
def generateMotifs(k, nM):
    return [generateRandomSequences(k, True) for rs in range(nM)]

def gen_n_substitMotifs(v, k, t):
    motifs = generateMotifs(k, 1)
    print(motifs[0])
    motif = ''.join(motifs[0])
    motifs = []
    i=0
    while i < t:
        j = 0
        motif_a_modif = list(motif.strip())
        while j < v:
            motif_a_modif[random.randint(0, k - 1)] = nuc[random.randint(0,3)]
            motif_add = ''.join(motif_a_modif)
            j+=1
        motifs.append(motif_add)
        i+=1
    return motifs
    
def implantMotifVar(k, v, t, n):
    #sequences aleatoires
    sequences = generateRandomSequences(n, t)
    if v != 0:
        motifs = gen_n_substitMotifs(v, k, t)
    else:
        motifs_generated = generateMotifs(k, 1)
        # il faut remettre dans une simple liste vu QUE GENERATE_MOTIFS fait une liste de LISTE
        motifs = motifs_generated[0]
    modified_sequences = []
    i = 0
    while i < t:
        seq = sequences[i]
        if v != 0:
            motif = motifs[i]
        else:
            motif = motifs[0]
        rand_pos = random.randint(0,len(seq))
        seq = seq[0:rand_pos] + motif + seq[rand_pos+1:len(seq)]
        modified_sequences.append(seq)
        i+=1
    return motifs, modified_sequences


In [3]:
#generate and implant variavel motifs
motifs_generated_seq, sequences = implantMotifVar(k, v, t, n)
print(motifs_generated_seq)
print(len(motifs_generated_seq))
print("sequences")
print (sequences)

['TAATGCT']
['TAACCAT', 'TAACATG', 'TAATAAA', 'TAAGCAT', 'TGATGCT', 'TATTGCT', 'TAAAACC', 'TGCTGCA', 'AAAAGCC', 'TAATGCT', 'TCATGGC', 'TAGGGCA', 'TGATTCA', 'TTGGGCT', 'TTATCAT', 'CGATGCT', 'TAATTCC', 'TATTACG', 'TGAAGCT', 'CGTTGCT']
20
sequences
['CGCGCCGTAACCATCATTCGTTAGTGCCTGGATGACCCCATATTGCCGTTCGGTAA', 'AGTCCTCCTTGCATGCAGGATGCGAGTTTTACACTGATAACATGGCGTGAGTAATA', 'TCTTGGAGCCAGACTCCCTTAAGAAACTAATAAATTGGACAGTGACATAACTCTCA', 'ATCAACGCGTCTAACAGCTGAATGATTCGACTCAGCTGGGACAAGGTGTAAGCATT', 'GATATAGGCAGCCTATGGGCAATTACCCTGATGCTGAAATGACAAGTCCACAGGTT', 'CCCACTATTGTCAGTGGGAGCGCTGTACTATCATTCCTATTCTCTTATTGCTCTGC', 'GGCTCAGTGGCGCGCTGTATCCATGGGTAAAACCGCTACGTCTCCGTGCTTATGTC', 'ACTGCTGCAAAAGTTTGTGTCAAAACCTCTAGGTGGAAACAATTACAGTACCTCAT', 'GCAGACGCGACTGTCTGTCAAAAGCCTAATAGAGACTGCCCTGAGAATATCGCGGA', 'CGTCCCTTAATGCTACGGTTTCTAGTGTGGTTCAACTCCATGGTGATCCTTTCATG', 'TGACTCATTGGTCCAGCATGTAGAGCAAAACAACATACCTTGTGCTTCATGGCGAT', 'TGCTAGGGCAAGGCCTCTAGTGGATTGCGCAAATTGCAAGTCGCGCCGCGAGACCT', 'GTAAATTCACAGAACGATTCTCGGCTGATTCA

2\. Faire une fonction pour sélectionner des positions de départ aléatoirement s = (s1, …,st).


In [4]:
#creating vector s
#Get t random positions from 0 to n - k
def generateRandomS(sequences,t,n,k):
	s = []
	for i in range(t):
		s.append(random.randint(0,n-k))
	return (s)
	#return random.sample(range(0,n-k),t)   Celui ci evite les doublons mais ne fonctionne pas quand le nombre de sequences est supérieur au nombre de bases par 												sequence (donc il ne fonctionne pas lorsque l'on teste avec notre châine)
s = generateRandomS(sequences,t,n, k)
print (s)

[37, 20, 39, 17, 26, 10, 18, 41, 40, 6, 21, 22, 4, 4, 10, 4, 11, 22, 10, 3]


3\. Extraire les motifs en utilisant le vecteur s, et construire un profile. Attention, utiliser les pseudocount 1 pour éviter les probabilités zéro.


In [5]:
from collections import Counter
m = 4

def removeLowComplexe(motifs, minrep):
    validMotif = []
    for motif in motifs:
        frequence = Counter(motif)
        # temoin pour savoir si une lettre est repetee ou non
        temoin = True
        for lettre, value in frequence.items():
            if value >= minrep:
                temoin = False
        if temoin == True:
            validMotif.append(motif)
                
    return validMotif

def removeLowComplexePair(motifs):
    validMotif = []
    for motif in motifs:
       frequence = Counter(motif)
       if len(frequence) > 2:
           validMotif.append(motif)

    return validMotif


In [6]:

#extract sequences from seqs
def extractSeqs(s, seqs):
    motifs = []    #Liste de motifs retour
    for start in s:     #on parcourt les positions départ
        for seq in seqs:    #et les sequences
            motifs.append(seq[start:start+k])   #on ajoute a la liste retour les k bases a partir de start

    #on enleve les motifs non interessants
    #print(len(motifs))
    motifs = removeLowComplexe(motifs, m)
    motifs = removeLowComplexePair(motifs)
    #print(len(motifs))
    return motifs
        
motifs = extractSeqs(s, sequences)
print(motifs)

#construct Profiles
def profile(motifs, k, t):
    q = len(nuc)
    res = np.ones((q, k))#pseudo count
    for motif in motifs:
        i = 0
        while i < len(motif):
            l = motif[i]
            if l == 'A':
                res[0][i] += 1
            if l == 'C':
                res[1][i] += 1
            if l == 'G':
                res[2][i] += 1
            if l == 'T':
                res[3][i] += 1
            i += 1
    return res
P = profile(motifs,k,t)
print(P)


['CCATATT', 'TAACATG', 'GACAGTG', 'GGGACAA', 'ACGTCTC', 'GCCCTGA', 'CCATGGT', 'CCTTGTG', 'AAGTCGC', 'GCATTCA', 'CCTGGTG', 'CCAGTGG', 'CATCTTG', 'TCCAGCG', 'GGTTGAC', 'TTAGTGC', 'TGCGAGT', 'AATGATT', 'AATTACC', 'CGCTGTA', 'CCATGGG', 'AAAGCCT', 'TCTAGTG', 'TAGAGCA', 'TGGATTG', 'TCGGCTG', 'GAGTACC', 'ACAGTTG', 'CGGATCC', 'TGAAGCT', 'GATACGC', 'ATATTGC', 'ACATGGC', 'CAGTGAC', 'GACAAGG', 'TGACAAG', 'GTCTCCG', 'CAATTAC', 'CCTGAGA', 'ATGGTGA', 'GTCGCGC', 'ATTCAAG', 'AGTGGAT', 'TCTTGGA', 'CAGCGTG', 'TTGACGT', 'TCGTTAG', 'CTTAAGA', 'CTGAATG', 'GGCAATT', 'GAGCGCT', 'TATCCAT', 'TGTCAAA', 'ATGTAGA', 'TAGTGGA', 'TTCTCGG', 'CTGCTCG', 'CATGAGT', 'ACCACAG', 'GAACGGA', 'AGGTGAA', 'CCTGGAT', 'TCGACTC', 'CCTGATG', 'ACTATCA', 'CTCTAGG', 'GTGGTTC', 'GCGCAAA', 'GATTCAC', 'CGGCCTA', 'GTACGAT', 'ACGTTCG', 'TCCAGTT', 'CACTTAC', 'CCATCAT', 'GCATGCA', 'AGACTCC', 'CTAACAG', 'GCCTATG', 'TCAGTGG', 'CGCGCTG', 'AAGTTTG', 'CTGTCTG', 'TGCTACG', 'GTCCAGC', 'AGGCCTC', 'CGATTAT', 'TTGTCGG', 'TTCGCAG', 'CTGCATT', 'GATCGGT'

4\. Transformer le Profile en PWM. Il faut diviser chaque élément par la somme de ses colonnes.

In [7]:
def getPWM(P):
    res=[]
    i=0
    j=0
    while i<4:  #Il y aura toujours que 4 bases (A,C,G,T) 
        tempres=[]
        while j<k: #k etant la longueur des motifs, c'est aussi la longueur des profils
            tempres.append(P[i][j]/(P[0][j]+P[1][j]+P[2][j]+P[3][j]))
            j+=1
        j=0
        i+=1
        res.append(tempres)
    return res

_PWM=getPWM(P)
print(_PWM)

[[0.25559105431309903, 0.23003194888178913, 0.24920127795527156, 0.18210862619808307, 0.20447284345047922, 0.24281150159744408, 0.22364217252396165], [0.25559105431309903, 0.25559105431309903, 0.19169329073482427, 0.26517571884984026, 0.28434504792332266, 0.2268370607028754, 0.24600638977635783], [0.23961661341853036, 0.2523961661341853, 0.3035143769968051, 0.24281150159744408, 0.2747603833865815, 0.24281150159744408, 0.31629392971246006], [0.24920127795527156, 0.26198083067092653, 0.25559105431309903, 0.30990415335463256, 0.2364217252396166, 0.28753993610223644, 0.21405750798722045]]


5\. Faire une fonction pour calculer la probabilité d'un motif de taille k selon une PWM.


In [8]:
def probability(Pwm, motif):
    prob = 1
    i=0
    while i<k:
        base=motif[i]
        #A est sur la première ligne du profil, C 2, G 3 et T 4
        if base=='A':
            prob=prob*Pwm[0][i]
        if base=='C':
            prob=prob*Pwm[1][i]
        if base=='G':
            prob=prob*Pwm[2][i]
        if base=='T':
            prob=prob*Pwm[3][i]
        i+=1
    return prob


6\. Faire une fonction pour calculer le pMostProbkmer d'une séquence, voir un exemple dans les slides de cours.

In [9]:
def pMostProbkmer(Pwm, k, sequence):
    maxProb = 0
    s = 0 # position du k-mer le plus probable dans la sequence
    i = 0 
    while i<len(sequence)-k: #on teste que des sequences de taille k
        iprob=probability(Pwm,sequence[i:i+k])
        if maxProb<iprob: 
            s=i
            maxProb=iprob
        i+=1
    return s

print(pMostProbkmer(_PWM,k,sequences[0]))

23


7\. Faire une fonction pour obtenir les nouvelles positions de départ s = (s1, …,st), c’était à dire les positions qui contiens les k-mer le plus probables.

In [10]:
def getNewS(Pwm, k, sequences):
    s = []
    for seq in sequences:
        s.append(pMostProbkmer(Pwm,k,seq)) #pMostProbKmer renvoiela position du kmer le plus probable de la sequence
    return s

ns = getNewS(_PWM, k, sequences)
print (ns)

[23, 42, 5, 43, 12, 7, 37, 12, 10, 22, 37, 45, 20, 10, 38, 34, 4, 27, 39, 32]


8\. La condition d’arrêt d’algorithme est le non changement du Profile d'une itération à l'autre, faire une fonction pour comparer deux Profils (matrice) et détecter le changement.

In [11]:
def changeProfile(Profile1, Profile2):
    i=0
    j=0
    while i<4:
        while j<k:
            if Profile1[i][j]!=Profile2[i][j]: #si une case diffère d'un profile a l'autre alors on renvoie True
                return True
            j+=1
        j=0
        i+=1
    return False #sinon les deux profils sont egaux donc on n'avance plus

corresp = {0:"A", 1:"C", 2:"G", 3:"T"}

def getScore(Pr, k):
    sc = 0
    i = 0
    j = 0
    base = ''
    consensus = ""
    while j < len(Pr[i]):
        max_ = 0
        while i < len(Pr):
            if Pr[i][j] > max_:
                max_ = Pr[i][j]
                base = corresp[i]
            i += 1
        sc += max_
        consensus += base
        i = 0
        j += 1
    return consensus, sc

ns = getNewS(_PWM, k, sequences)
print (ns)

consensus, sc = getScore(P, k)
print (consensus)
print (sc)


[23, 42, 5, 43, 12, 7, 37, 12, 10, 22, 37, 45, 20, 10, 38, 34, 4, 27, 39, 32]
ATGTCTG
632.0


9\. Implementer l'algorithme ``GreedyProfileMotifSearch`` en utilisant les fonctions precedentes. 


In [12]:
import time
def GreedyProfileMotifSearch(sequences, t, n, k):
    #resultats
    bestScore = 0
    consensus = ""
    #selection de positions starts
    starts = generateRandomS(sequences, t, n, k)
    #creation des motifs avec les starts
    motifs = extractSeqs(starts, sequences)
    #creation du profile associé aux starts
    profil = profile(motifs, k, t)
    #creation du PWM
    pwm = getPWM(profil)
    #calcul du score
    cons, score = getScore(profil, sequences)

    cpt = 0
    while score > bestScore:
        cpt += 1
        #attribution nouveau score et nouveau consensus
        bestScore = score
        consensus = cons

        #generation de nouveaux starts censés etre meilleurs
        starts = getNewS(pwm, k, sequences)
        #creation des motifs avec les starts
        motifs = extractSeqs(starts, sequences)
        #creation du profile associé aux starts
        profil = profile(motifs, k, t)
        #creation du PWM
        pwm = getPWM(profil)
        #calcul du score
        cons, score = getScore(profil, sequences)

    return cons, bestScore, starts

start = time.time()
consensus, s, bestScore = GreedyProfileMotifSearch(sequences, t, n, k)  
fin = time.time()
print("temps = {}".format(fin-start))
print(consensus) 
print("motifs implantés : ")
print(motifs_generated_seq)
print(s, bestScore)

temps = 0.010501623153686523
CCTTCTG
motifs implantés : 
['TAACCAT', 'TAACATG', 'TAATAAA', 'TAAGCAT', 'TGATGCT', 'TATTGCT', 'TAAAACC', 'TGCTGCA', 'AAAAGCC', 'TAATGCT', 'TCATGGC', 'TAGGGCA', 'TGATTCA', 'TTGGGCT', 'TTATCAT', 'CGATGCT', 'TAATTCC', 'TATTACG', 'TGAAGCT', 'CGTTGCT']
613.0 [13, 5, 30, 31, 26, 10, 3, 0, 39, 38, 4, 16, 43, 31, 5, 6, 37, 8, 27, 35]


10\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

Reponse: nous trouvons le motif CTCAATT, le motif le plus proche que nous avons généré est CTAACTT. Il etait peu probable d'arriver a retrouver ce motif ou un autre étant donné le nombre important de bases mutées dans 
nos motifs (4 sur 7...)

11\. Vous avez certainement observer que l’algorithme ne produire pas toujours la même sortie et que les résultats dépendent de la sélection aléatoire des positions de départ s. Pour augmenter nous chances de retrouvez les bons motifs, nous allons implémenter une version itérative ``GreedyProfileMotifSearchIte`` 
qui a chaque fois sauvegarde s et le score du profile associé à s, après I itération, l’algorithme renvoie le vecteur s ayant le plus grand score.

In [13]:

def GreedyProfileMotifSearchIte(sequences, t, n, k, It):
    ScoreList=[]
    StartList=[]
    ConsensusList = []
    i = 0
    while i < It:
        consensus, score, starts = GreedyProfileMotifSearch(sequences, t, n, k)
        ConsensusList.append(consensus)
        ScoreList.append(score)
        StartList.append(starts)
        i += 1
        #print(i)

    index = ScoreList.index(max(ScoreList))
    return ConsensusList[index], ScoreList[index], StartList[index]

start=time.time()
consensusIte, bestScoreIte, startsIte = GreedyProfileMotifSearchIte(sequences, t, n, k, 50)
end=time.time()
print("temps : {}".format(end-start))
print (consensusIte)
print(motifs_generated_seq)
print (bestScoreIte)
print (startsIte)

temps : 0.6226096153259277
CCTGTTG
['TAACCAT', 'TAACATG', 'TAATAAA', 'TAAGCAT', 'TGATGCT', 'TATTGCT', 'TAAAACC', 'TGCTGCA', 'AAAAGCC', 'TAATGCT', 'TCATGGC', 'TAGGGCA', 'TGATTCA', 'TTGGGCT', 'TTATCAT', 'CGATGCT', 'TAATTCC', 'TATTACG', 'TGAAGCT', 'CGTTGCT']
699.0
[38, 15, 16, 31, 26, 3, 47, 0, 6, 38, 37, 15, 20, 13, 41, 37, 25, 1, 27, 35]


12\. Tester algorithme  ``GreedyProfileMotifSearchIte`` sur vos données de chipSeq.

In [14]:
k=7; t=20; n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "./Sequence_by_Peaks_6.fasta" #path of your fasta seqs

sequencesChip = readFasta(genome, n)
t = len(sequencesChip)
print (t, n, k)

consensusIteChip, bestScoreIteChip, startsIteChip = GreedyProfileMotifSearchIte(sequencesChip, t, n, k, 50)
print (consensusIteChip, bestScoreIteChip, startsIteChip)

330 80 7


KeyboardInterrupt: 

In [15]:
#Nous eliminons les motifs peu interessants avec les deux fonctions removelowcomplexe et removepair dans extractseqs mais le consensus est la construction d'un motif a partir du profile donc on ne peut pas #controler.
#Nous pouvons cependant parcourir les sequences en prenant a chaque fois le motif associé a sa position de depart pour avoir la liste de motifs resultats de l'algo. On pourra ensuite filtrer cette liste a l'aide
#de nos deux fonctions

In [16]:
m = 5

motifsChip = []
i = 0
for s in startsIteChip:
    seq = sequencesChip[i]
    motifsChip.append(seq[s:s+k])
    i+=1

motifsChip = removeLowComplexe(motifsChip, m)
motifsChip = removeLowComplexePair(motifsChip)
print(motifsChip)

NameError: name 'startsIteChip' is not defined

13\. Nous allons implémenter l'algorithme ``randomProjection``. Faire une fonction pour générer une projection de l à k, voir un exemple dans les slides de cours

In [17]:
k=7; v=1; t=20; n=50

def getRandomFixePositions(k, n):
    return random.sample(range(n), k)

lR = getRandomFixePositions(k, n)
lR.sort()
print (lR)

def generateKey(positions, motif):
    m = ""
    for i in positions:
        m = m + motif[i]
    return m
key = generateKey(lR, sequences[0])
print(key)


[8, 21, 27, 34, 39, 42, 47]
ATCAATG


14\. Implémenter l'algorithme ``randomProjection``.

In [18]:
import time

def randomProjection(k, n, sequences, motifs, motifsSeq):
    motifs  = {} 
    motifsSeq  = {}

    lR = getRandomFixePositions(k, n)
    lR.sort()
    for seq in sequences:
        motif = generateKey(lR, seq)
        #ajout dans motifs
        if motif not in motifs:
            motifs[motif] = 1
        else:
            motifs[motif] += 1
        #ajout dans motifsSeq
        motifsSeq[seq] = motif

    return motifs, motifsSeq

start  = time.time()
motifs, motifsSeq = randomProjection(k, n, sequences, {}, {})
end=time.time()
motifsSort = sorted(motifs, reverse=True, key=motifs.get)
print(end-start)
print (motifsSort)
print(motifs) 

0.0
['GTCTGCG', 'CCGGGAT', 'AGTAAGT', 'GCCAGTG', 'GGAATAC', 'ATTGTCT', 'GTCCTTG', 'GCTAAAG', 'GCCAGTT', 'TTACATC', 'ATAAAAC', 'GGCGACG', 'TCCCGCA', 'GACTGTT', 'ACTATAG', 'TTCCGGT', 'TTCGATG', 'TGCAAAG', 'TAAGATA', 'TGGAACG']
{'GTCTGCG': 1, 'CCGGGAT': 1, 'AGTAAGT': 1, 'GCCAGTG': 1, 'GGAATAC': 1, 'ATTGTCT': 1, 'GTCCTTG': 1, 'GCTAAAG': 1, 'GCCAGTT': 1, 'TTACATC': 1, 'ATAAAAC': 1, 'GGCGACG': 1, 'TCCCGCA': 1, 'GACTGTT': 1, 'ACTATAG': 1, 'TTCCGGT': 1, 'TTCGATG': 1, 'TGCAAAG': 1, 'TAAGATA': 1, 'TGGAACG': 1}


15\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

reponse: oui

16\. Implémenter la version itérative de l’algorithme ``randomProjection``. 

In [19]:
def randomProjIt(sequences, k, n, It):
    motifs  = {} 
    motifsSeq  = {}
    i = 0
    while i < It:
        #on appelle randomProjection
        motifs, motifsSeq = randomProjection(k, n, sequences, motifs, motifsSeq)  
        i += 1
    return motifs, motifsSeq

score, seqsMotif = randomProjIt(sequences, k, n, 100)
print (score)
print(seqsMotif)


{'AATGGTA': 1, 'TCTTCCA': 1, 'CCAAAAG': 1, 'GAAAAAC': 1, 'CTAGCGA': 1, 'TGCATTC': 1, 'GTCACTC': 1, 'ATCTTAA': 1, 'GTATGCT': 1, 'ACTGATG': 1, 'TGTACTG': 1, 'CTTCTTC': 1, 'AGTTTTT': 1, 'TTCCTTT': 1, 'GAGCCGC': 1, 'TGACGGG': 1, 'TACTAGT': 1, 'TAATGCT': 1, 'CTTAGAG': 1, 'TGGTTTG': 1}
{'CGCGCCGTAACCATCATTCGTTAGTGCCTGGATGACCCCATATTGCCGTTCGGTAA': 'AATGGTA', 'AGTCCTCCTTGCATGCAGGATGCGAGTTTTACACTGATAACATGGCGTGAGTAATA': 'TCTTCCA', 'TCTTGGAGCCAGACTCCCTTAAGAAACTAATAAATTGGACAGTGACATAACTCTCA': 'CCAAAAG', 'ATCAACGCGTCTAACAGCTGAATGATTCGACTCAGCTGGGACAAGGTGTAAGCATT': 'GAAAAAC', 'GATATAGGCAGCCTATGGGCAATTACCCTGATGCTGAAATGACAAGTCCACAGGTT': 'CTAGCGA', 'CCCACTATTGTCAGTGGGAGCGCTGTACTATCATTCCTATTCTCTTATTGCTCTGC': 'TGCATTC', 'GGCTCAGTGGCGCGCTGTATCCATGGGTAAAACCGCTACGTCTCCGTGCTTATGTC': 'GTCACTC', 'ACTGCTGCAAAAGTTTGTGTCAAAACCTCTAGGTGGAAACAATTACAGTACCTCAT': 'ATCTTAA', 'GCAGACGCGACTGTCTGTCAAAAGCCTAATAGAGACTGCCCTGAGAATATCGCGGA': 'GTATGCT', 'CGTCCCTTAATGCTACGGTTTCTAGTGTGGTTCAACTCCATGGTGATCCTTTCATG': 'ACTGATG', 'TGACTCA

17\. Tester l'algorithme  ``randomProjection`` sur vos données de chipSeq. Puis générér le LOGO du motif trouvé

In [24]:
k=7; n=80
sequencesChip = readFasta(genome, n)
score, seqsMotif = randomProjIt(sequencesChip[0:len(sequencesChip) - 2], k, n, 100)
#print (score)
#print(seqsMotif)
liste_motif = []
with open('motifsRandProj.fasta', 'w') as file :
    for seq, motif in seqsMotif.items():
        liste_motif.append(motif)
        #print(motif)
        #file.write(motif+"\n")

GTGTAGC
TGCGGAA
CAGGTCA
AAAACAT
CGAACAA
AACCTCC
AACACTC
GCATTAC
ATAATGT
GCGATTG
AATCATA
GTTACTC
TAGAATC
ATTTGAC
TTGGTTG
TGAGGAA
CGCCTTT
ATCTAGT
CCTTGAA
AAGAACA
TTCCTTG
TGAATAT
GAGAATA
TCTATGC
AGTAACC
TGTCAAA
CAATTTC
GCAGCAT
GTTCAAG
TTTAGAG
GTTTCTT
CCCTATC
ATTCGTC
CGTTGCG
AGACCTC
AAGGTGA
GCCCTCC
TCCGAGG
GAGTATA
GGGGGAG
CAATTGT
AATCTGT
TCCAGTG
ATGCATT
ATACACC
TCCGCAG
ATATGAG
ATGTAAG
TAATTGT
TGTACTG
TTTGGGT
GATGTAA
TTGATAC
CTCTGAG
ATTAAAA
GCACGTG
TTCAGTT
CTAGATT
AGTATAT
CTCAATG
CCAGTAT
GCGGTCC
GATAACG
CGGTTAC
TATCAAT
CTCCCTA
TCGGTCA
ATATGCA
CTCGCAA
TGCGAAA
GCCAAAG
TACTGTG
TAATAGT
AACTCCC
CCCTGGC
CTATAAG
GAGGCTA
CTATAAA
ATATCGT
TTATCAT
TGTTGGA
AATCCCA
CCCTTCA
TTCTAAG
TCTGGAG
CTAGATT
ACTTGAG
AAGCGGT
TAGATAA
TCAGCGA
GACTCAT
AATAAAA
GTGGCGA
TGAGACG
GCTTCGC
ATTCGTC
GTTCTCT
CATATGT
CGTATGA
GGTCGTA
TCGGGCG
CTGTCTA
AAAGATC
GCTGACG
TGGGCTG
AGGGCGA
GAGGGGT
TGAAAAG
AACAAGC
GTATATA
GGAGTCA
CCCATAG
CTCCGTT
ATGCATG
TTTAACG
TAAGTAC
AGCATTG
GACGTTA
TTGAGAC
TATGGAA
GCTATAC
GAAAAAG
TATTTGC
ATGGCTG
TAGTAAG
